In [1]:
from llama2 import *
from typing import List, Literal, Optional, Tuple, TypedDict
from pprint import pprint

/home/chris/Desktop/repos/llama2/.env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Llama2ChatModel(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    model_resolution='int4'
)

model.model

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [3]:
dialogs: List[Dialog] = [
    [Message(role='user', content='Briefly explain the difference between pandas and pyspark')],
]

dialogs = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
    ]

In [4]:
dialogs = model.preprocess_dialog(dialogs)
pprint(dialogs)

['<s>[INST] <<SYS>>\n'
 '    You are a helpful, respectful and honest assistant. Always answer as '
 'helpfully as possible, while being safe. Your answers should not include any '
 'harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. '
 'Please ensure that your responses are socially unbiased and positive in '
 'nature.\n'
 '\n'
 '    If a question does not make any sense, or is not factually coherent, '
 "explain why instead of answering something not correct. If you don't know "
 "the answer to a question, please don't share false information.\n"
 '<</SYS>>\n'
 '\n'
 'what is the recipe of mayonnaise? [/INST]',
 '<s>[INST] <<SYS>>\n'
 '    You are a helpful, respectful and honest assistant. Always answer as '
 'helpfully as possible, while being safe. Your answers should not include any '
 'harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. '
 'Please ensure that your responses are socially unbiased and positive in '
 'nature.\n'
 '\n'
 '

In [5]:
model.tokenizer.add_special_tokens({'pad_token': '[PAD]'})
llm = model.model

In [11]:
x[0, :]

tensor([    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,  1678,
          887,   526,   263,  8444, 29892,  3390,  1319,   322, 15993, 20255,
        29889, 29849,  1234,   408,  1371,  3730,   408,  1950, 29892,  1550,
         1641,  9109, 29889,  3575,  6089,   881,   451,  3160,   738, 10311,
         1319, 29892,   443,   621,   936, 29892, 11021,   391, 29892,  7916,
          391, 29892,   304, 27375, 29892, 18215, 29892,   470, 27302,  2793,
        29889,  3529,  9801,   393,   596, 20890,   526,  5374,   635,   443,
         5365,  1463,   322,  6374,   297,  5469, 29889,    13,    13,  1678,
          960,   263,  1139,   947,   451,  1207,   738,  4060, 29892,   470,
          338,   451,  2114,  1474, 16165,   261,   296, 29892,  5649,  2020,
         2012,   310, 22862,  1554,   451,  1959, 29889,   960,   366,  1016,
        29915, 29873,  1073,   278,  1234,   304,   263,  1139, 29892,  3113,
         1016, 29915, 29873,  6232,  2089,  2472, 29889,    13, 

In [7]:
tokens = model.tokenizer(dialogs, padding=True, add_special_tokens=False, return_tensors='pt')
x = tokens['input_ids']
# x[x==32000] = -1
x = x.cuda()

llm.generate(x[0, :], num_return_sequences=1, generation_config=model.generation_config)

ValueError: not enough values to unpack (expected 2, got 1)

In [6]:
model.tokenizer.add_special_tokens({'pad_token': '[PAD]'})

output = model.generate(dialogs[0])

/home/chris/Desktop/repos/llama2/.env/lib/python3.11/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
print(output)

<s> [INST] <<SYS>>
    You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

what is the recipe of mayonnaise? [/INST]  Thank you for reaching out! I'm happy to help you with your question. However, I must inform you that mayonnaise is a complex condiment that requires a specific set of ingredients and techniques to prepare. It's not something that can be easily explained in a recipe format, as the proportions and methods of preparation can vary greatly depending on the individual or the recipe.
Mayonnaise is typically made by

In [ ]:
del output